## Imports

In [1]:
from datasets import load_dataset

In [4]:
!ls -lah datasets

total 6.7M
drwxrwsr-x 4 am3052 am3052 4.0K Jun 14 23:54 .
drwxrwsr-x 3 am3052 am3052 4.0K Jun 14 23:55 ..
-rw-rw-r-- 1 am3052 am3052 2.8M Jun 14 16:08 burns_datasets_VINC_train.parquet
-rw-rw-r-- 1 am3052 am3052 858K Jun 14 23:54 burns_datasets_VINC_validation.parquet
drwxrwsr-x 9 am3052 am3052 4.0K Jun 14 22:18 ppo_dataset_raw
-rw-rw-r-- 1 am3052 am3052 3.0M Jun 14 23:53 ppo_training_dataset.parquet
drwxrwsr-x 9 am3052 am3052 4.0K Jun 14 22:18 test_dataset_raw


In [13]:
# parquet_data_files = {
#     "train": "datasets/burns_datasets_VINC_train.parquet",
#     "validation": "datasets/burns_datasets_VINC_validation.parquet",
# }

parquet_data_files = {
    "train": "datasets/ppo_training_dataset.parquet",
}

my_dataset = load_dataset("parquet", data_files=parquet_data_files)
my_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /home/am3052/.cache/huggingface/datasets/parquet/default-069d4dfb5d344d67/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['original_dataset', 'template_name', 'prompt', 'best_response'],
        num_rows: 10122
    })
})

In [14]:
# my_dataset.push_to_hub("AugustasM/burns-ppo-training-dataset", private=True)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]